In [23]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup








In [ ]:
chrome_driver_path = r"C:\Users\loges\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"


options = Options()
options.add_argument("--headless")
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
url = "https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021"
driver.get(url)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()


overs = soup.select("div.cb-col.cb-col-8.text-bold.cb-com-ln")
comments = soup.select("div.cb-col.cb-col-100.cb-com-ln")

over_data = []
for over, comment in zip(overs, comments):
    over_num = over.get_text(strip=True)
    comment_text = comment.get_text(strip=True)
    
    over_data.append({
        "over": over_num,
        "comment": comment_text
    })




In [24]:
df_commentary = pd.DataFrame(over_data[:30])  
df_overinfo = pd.DataFrame(over_data[30:60])  

df_commentary.to_csv("commentary_data.csv", index=False)
df_overinfo.to_csv("over_data.csv", index=False)

print("Scraped and saved two datasets!")

Scraped and saved two datasets!


In [25]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("UnionExample") \
    .master("local[*]") \
    .getOrCreate()




In [27]:
df_commentary = spark.read.option("header", True).csv("commentary_data.csv")
df_overinfo = spark.read.option("header", True).csv("over_data.csv")




In [28]:
print("Commentary Data:")
df_commentary.show(5)


Commentary Data:
++
||
++
++



In [29]:
print("Over Info Data:")
df_overinfo.show(5)


Over Info Data:
++
||
++
++



In [30]:
union_df = df_commentary.union(df_overinfo)
print("Combined using union():")
union_df.show(5)

Combined using union():
++
||
++
++



In [33]:

from pyspark.sql.functions import col, lit


df_overinfo_extra = df_overinfo.withColumn("source", lit("over_data"))
df_commentary_extra = df_commentary.withColumn("source", lit("commentary"))




In [35]:
union_by_name_df = df_commentary_extra.unionByName(df_overinfo_extra)


In [37]:
print("Combined using unionByName():")
union_by_name_df.show(5)



Combined using unionByName():
+------+
|source|
+------+
+------+

